In [42]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Predefined list of skills and locations
skills = ['intern']
locations = ['Boston', 'New York', 'San Fransisco', 'Los Angeles', 'Denver', 'Seattle']
pages = 5 # pages per distinct combo of skills/location

# Set driver
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)


jobs_data = []
# Scraping the Web with Selenium for each skill and location combination
for skill in skills:
    for place in locations:
        for page in range(pages):  # Scrape specified number of pages
            base_url = f'https://www.indeed.com/jobs?q={skill}&l={place}&start='
            url = base_url + str(page * 10)
            driver.get(url)

            # Let the page load
            time.sleep(1)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # All job cards
            job_containers = soup.find_all('div', class_='tapItem')
            
            # Extract info and append to CSV
            for job in job_containers:
                job_title_element = job.find('h2', class_='jobTitle')
                job_title = job_title_element.find('span').text.strip() if job_title_element else 'Title not available'

                company_element = job.find('span', class_='css-1x7z1ps')
                company_name = company_element.text.strip() if company_element else 'Company name not available'

                location_element = job.find('div', class_='css-t4u72d')
                location = location_element.text.strip() if location_element else 'Location not available'

                salary_element = job.find('div', class_='metadata salary-snippet-container')
                salary = salary_element.find('div', class_='css-1ihavw2').text.strip() if salary_element else 'Salary not available'

                job_description = job.find('div', class_='job-snippet').text.strip() if job.find('div', class_='job-snippet') else 'Description not available'

                jobs_data.append([job_title, company_name, location, job_description, salary])

# Quit the WebDriver
driver.quit()

# Read the existing jobs.csv file as a DataFrame
existing_jobs = pd.read_csv('jobs.csv')

# Convert the scraped data into a DataFrame
new_jobs_df = pd.DataFrame(jobs_data, columns=['JOB_NAME', 'COMPANY', 'LOCATION', 'DESCRIPTION', 'SALARY'])

# Concatenate the existing and new job data, drop duplicates, and re-save to jobs.csv
all_jobs = pd.concat([existing_jobs, new_jobs_df]).drop_duplicates().reset_index(drop=True)
all_jobs.to_csv('jobs.csv', index=False)

print(str(len(all_jobs) - len(existing_jobs)) + ' jobs were found and added to jobs.csv successfully.')


444 jobs were found and added to jobs.csv successfully.
